In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
# read in vibs for each surface in Eform from csv, replace NaN with 0
def read_vibs(sysname,Nform):
    df = pd.read_csv(sysname)
    df2 = df.fillna(0)
    vibs = []
    for ii in range(0,Nform,1):
        vibs.append(list(df2.iloc[:,ii]))
    return vibs

In [ ]:
# calculate trans partition function
def qtrans(mass,T,P): # mass input should be in kg/molec, partial pressure input should be in Pa
    kb = 1.380649*10**-23 # J/K, Boltzmann's constant
    h = 6.62607015*10**-34 # Js, Planck's constant
    qout = ((2*np.pi*mass*kb*T/h**2)**1.5)*(kb*T/P)
    return qout

In [ ]:
# calculate diatomic rot partition function
def qrot_di(red_mass,R,T,sig): # reduced mass input should be in kg/molec, bond distance input should be in m
    kb = 1.380649*10**-23 # J/K, Boltzmann's constant
    h = 6.62607015*10**-34 # Js, Planck's constant
    I = red_mass*R**2
    qout = 8*(np.pi**2)*I*kb*T/(sig*h**2)
    return qout

In [ ]:
# calculate polyatomic rot partition function
def qrot_poly(Iabc,T,sig):
    kb = 1.380649*10**-23 # J/K, Boltzmann's constant
    h = 6.62607015*10**-34 # Js, Planck's constant    
    qout = (8*np.pi**2/(sig*h**3))*(2*np.pi*kb*T)**1.5*Iabc**0.5
    return qout

In [ ]:
# calculate vibs partition function
def qvib(vibs,T): # vibs input should be list with units of meV
    kB = 8.617333262*10**-5 # eV/K, Boltzmann's constant
    qout = 1
    value = 10
    for item in vibs:
        if item != 0:
            if item > value:
                qout = qout*(np.exp(-item/(2000*kB*T))/(1-np.exp(-item/(1000*kB*T))))
            else:
                qout = qout*(np.exp(-value/(2000*kB*T))/(1-np.exp(-value/(1000*kB*T))))
    return qout

In [ ]:
# calculate Gibbs free formation energy
def Gibbs_calc(Eform,vibs,Nform,n,P,T):
    kB = 8.617333262*10**-5 # eV/K, Boltzmann's constant
    
    # O2 constants
    mO2 = (31.99*10**-3)/(6.0221408*10**23) # kg, molar mass of gas phase O2
    redmO2 = mO2/4 # kg, reduced mass for O2
    R_O2 = 1.23456177009013*10**-10 # m, distance between both atoms in O2
    vO2 = [193.877146] # meV, vibs mode for O2
    
    Gtotal = []
    Pboltz = []
    for item in P: # loop pressure
        for item2 in T: # loop temperature
            save_jj = 0
            Pboltz_temp = []
            for ii in range(0,Nform):
                # Gas phase free energies               
                # oxygen (O2)
                Atrans_O2 = -kB*item2*np.log(qtrans(mO2,item2,item)) # eV
                Arot_O2 = -kB*item2*np.log(qrot_di(redmO2,R_O2,item2,2)) # eV
                Avib_O2 = -kB*item2*np.log(qvib(vO2,item2)) # eV
                
                # Surface free energy
                Avib_S = -kB*item2*np.log(qvib(vibs[ii],item2))

                # Total free energy
                Ggas = n[ii]/2*(Avib_O2 + Arot_O2 + Atrans_O2)
                Gtemp = Eform[ii] + Avib_S + Ggas
                Gtotal.append(Gtemp)
                
                # Boltzmann probability
                Pboltz_temp.append(np.exp(-(Gtemp-15)/kB/item2))
            sum_Pboltz_temp = sum(Pboltz_temp)
            temp = [thing/sum_Pboltz_temp for thing in Pboltz_temp]
            for thingthing in temp:
                Pboltz.append(thingthing)
    return Gtotal, Pboltz

In [ ]:
# ∆Gtotal for ab-Initio Phase Diagrams
plt.rc('xtick', labelsize=10)    # fontsize of the tick labels
plt.rc('ytick', labelsize=10)    # fontsize of the tick labels
plt.rc('axes', labelsize=12)    # fontsize of the x and y labels

################## user inputs ##################
npoints = 200
P = [10**-7,10**5] # Pa, Total pressure conditions
yO2 = 0.50 # O2 mole fraction
T = np.linspace(200,1600,npoints) # K, absolute temperature
n = [0,0,1,0,1,-1,-2,0,0,1,0,1,-1,-2] # number of Ovac for each system
# surf--no vac/Msurf/Osurf/Msub/Osub/Oads/O2ads 
# sub--no vac/Msurf/Osurf/Msub/Osub/Oads/O2ads
# Al 
Eform = [8.865073,13.758999,13.410649,14.085113,13.600143,8.211534,6.932308,
         8.727449,13.522155,13.40419,14.142579,13.205758,8.146524,6.758209]
sysname = 'Al-NiO100_dftu_vibs.csv'
#################################################

Nform = len(Eform)
vibs = read_vibs(sysname,Nform)
Gt, Pb = Gibbs_calc(Eform,vibs,Nform,n,P,T)
Gt_up, nope = Gibbs_calc([x+0.8 for x in Eform],vibs,Nform,n,P,T)
Gt_down, nope = Gibbs_calc([x-0.8 for x in Eform],vibs,Nform,n,P,T)

NP = 2 # number of pressure settings
Nn = Nform # number of configurations
NT = npoints # temperature range
N = len(Gt_up) // NP  # Calculate the size of each individual list

In [ ]:
fig = plt.figure(figsize=(8.5,8))
gs = fig.add_gridspec(2,2,wspace=0.12,hspace=0.12)
big_ax = fig.add_subplot(111)
# Turn off axis lines and ticks of the big subplot
big_ax.spines['top'].set_color('none')
big_ax.spines['bottom'].set_color('none')
big_ax.spines['left'].set_color('none')
big_ax.spines['right'].set_color('none')
big_ax.tick_params(labelcolor='w', top=False, bottom=False, left=False, right=False)
###################################################
ax = gs.subplots(sharey='row',sharex=True)
colors = ['red','blue','green','orange','purple','brown','pink','grey','olive','cyan','lime','teal','magenta','navy']
lines = ['-','-','-.','-','-.',':','--','-','-','-.','-','-.',':','--']
hatchs = ['ooo', '', '///', '', '\\\\\\', '++', 'xx', '****', '', '////', '', '\\\\\\\\\\', '+++', 'xxx']
alphas = 14*[0.4]
labels = ['Surf Perfect',r'Surf Ni$_{\rm surf}$',r'Surf O$_{\rm surf}$',r'Surf Ni$_{\rm sub}$',r'Surf O$_{\rm sub}$','O*/Surf Perfect',r'O$_{\rm 2}$*/Surf Perfect',
         'Sub Perfect',r'Sub Ni$_{\rm surf}$',r'Sub O$_{\rm surf}$',r'Sub Ni$_{\rm sub}$',r'Sub O$_{\rm sub}$','O*/Sub Perfect',r'O$_{\rm 2}$*/Sub Perfect']
labels_set = ['O$_2$ Lean','O$_2$ Rich']

# get upper and lower bounds for phase shades
GUHVu = []
GUHVu = Gt_up[:N]       # Extract the first N elements
GAMBu = []
GAMBu = Gt_up[N:2*N]    # Extract the next N elements

GUHVd = []
GUHVd = Gt_down[:N]       # Extract the first N elements
GAMBd = []
GAMBd = Gt_down[N:2*N]    # Extract the next N elements

PbUHV = []
PbUHV = Pb[:N]       # Extract the first N elements
PbAMB = []
PbAMB = Pb[N:2*N]    # Extract the next N elements
ref_max = -1000
ref_min = 1000
for jj in [0,1,2,3,4,5,6,7,8,9,10,11,12,13]:#range(0,Nn):
    GAu = []
    GBu = []
    GAd = []
    GBd = []
    PbA = []
    PbB = []
    for ii in range(0,NT):
        GAu.append(GUHVu[ii*Nn+jj])
        GBu.append(GAMBu[ii*Nn+jj])
        GAd.append(GUHVd[ii*Nn+jj])
        GBd.append(GAMBd[ii*Nn+jj])
        PbA.append(PbUHV[ii*Nn+jj])
        PbB.append(PbAMB[ii*Nn+jj])
        
    ax[0][0].fill_between(T,GAd,GAu,color=colors[jj],alpha=alphas[jj],hatch=hatchs[jj],label=labels[jj],edgecolor=None)
    ax[0][1].fill_between(T,GBd,GBu,color=colors[jj],alpha=alphas[jj],hatch=hatchs[jj],label=labels[jj],edgecolor=None)

    maxG = max(max(GAu),max(GBu))
    minG = min(min(GAd),min(GBd))
    if maxG > ref_max:
        ref_max = maxG
    if minG < ref_min:
        ref_min = minG
    
    ax[1][0].plot(T,PbA,label=labels[jj],color=colors[jj],linestyle=lines[jj])
    ax[1][1].plot(T,PbB,label=labels[jj],color=colors[jj],linestyle=lines[jj])

# Create legend with unique labels
handles, labels = ax[0][0].get_legend_handles_labels()
unique_handles, unique_labels = [], []
for handle, label in zip(handles, labels):
    if label not in unique_labels:
        unique_handles.append(handle)
        unique_labels.append(label)

ax[0][1].legend(handles=unique_handles, labels=unique_labels, fontsize=9, loc='center left', ncol=1, frameon=False, bbox_to_anchor=(1, 0.49))

handles, labels = ax[1][1].get_legend_handles_labels()
unique_handles, unique_labels = [], []
for handle, label in zip(handles, labels):
    if label not in unique_labels:
        unique_handles.append(handle)
        unique_labels.append(label)
ax[1][1].legend(handles=unique_handles, labels=unique_labels, fontsize=9, loc='center left', ncol=1, frameon=False, bbox_to_anchor=(1, 0.49))

####################

ax[0][0].set_title(labels_set[0],fontsize=12,color='black',weight="bold")
ax[0][1].set_title(labels_set[1],fontsize=12,color='black',weight="bold")

ax[1][0].set_ylabel('Boltzmann Probability', fontsize=12)  # Increase font size of y-axis label
ax[0][0].set_ylabel('ΔG$_{form}$ (eV)', fontsize=12)  # Increase font size of y-axis label
big_ax.set_xlabel('Temperature (K)', fontsize=12, labelpad=18)  # Increase font size of x-axis label

ax[0][0].set_ylim([ref_min-2,ref_max+2])
ax[1][0].set_yscale("log")  
ax[1][0].set_ylim([1e-14,2])
ax[1][0].set_xlim([min(T),max(T)])
ax[1][1].set_xlim([min(T),max(T)])

# Rotate the x-axis tick labels for all subplots
for a in [ax[1][0],ax[1][1]]:
    a.tick_params(axis='x',rotation=45)  # Adjust the rotation angle as needed

# plt.savefig("Al-NiO100_2Dpd_bp"+".png",bbox_inches='tight',transparent=False,dpi=600)
plt.show()